In [7]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import check_ranking
from googlesearch import search
import os
import datetime

class Search():
    
        def __init__(self,query="",domain="",filename=""):
            self.query = query
            self.domain = domain
            self.filename = filename
            
        def search_term(self):
            
            ## 日本語のGoogle検索を行う。言語は日本語。１ページあたりの検索数は10個,50個検索して止める。
            search_result_list = list(search(self.query, lang="jp", num=10, stop=50, pause=1))
            
            df = pd.DataFrame(search_result_list,columns=["urls"])

            ## 重複を消す
            clean_df = df[(~df.urls.str.contains("#"))].reset_index()

            ## ドメインを検索
            index = clean_df[clean_df.urls.str.contains(self.domain)].index.to_list() 

            ## インデックスに＋１をしてランキングにする
            rank = [n + 1 for n in index]

            dt_now = datetime.datetime.now()

            data = pd.DataFrame({'SearchTerm': self.query,
                                   'Ranking': [rank],
                                   'Total': len(rank),
                                   'TargetSite': self.domain,
                                   'Date':dt_now
                                   }
                                  )

            if not os.path.exists(self.filename):
                df = pd.DataFrame()
            else:
                df = pd.read_pickle(self.filename)

            df = df.append(data)
            
            df.reset_index(drop=True,inplace=True)
            
            print(df)
            df.to_pickle(self.filename)

            return
        
        def remove(self, index):
            pass
        
        def update(self, index):
            pass

class RankCheckerGUI:
    def __init__(self):
        self.filename = "ranking.pkl"
        self.domain = "ymgsapo.com"
        
        self.selected = []
        
        self.base = tk.Tk()
        self.base.title("RankChecker")
        self.base.geometry("800x800")
        
        #テキストボックス
        self.entryBox = tk.Entry(master=self.base)
        self.entryBox.pack()

        #検索ボタン
        self.button1 = tk.Button(
                master=self.base,
                text="Button",          #初期値
                width=15,                  #幅
                bg="lightblue",         #色
                command=self.query_click    #クリックに実行する関数
                )
        self.button1.pack()
        
        #削除ボタン
        self.button2 = tk.Button(
                master=self.base,
                text="Button",          #初期値
                width=15,                  #幅
                bg="lightblue",         #色
                command=self.remove_click    #クリックに実行する関数
                )
        self.button2.pack()
        
        # TreeViewの設定
        self.tree = ttk.Treeview(self.base)
        
        # 列を作る
        self.tree["column"] = (1, 2, 3, 4, 5,6)
        self.tree["show"] = "headings"

        # ヘッダーテキスト
        self.tree.heading(1, text="No.")
        self.tree.heading(2, text="検索ワード")
        self.tree.heading(3, text="ランキング")
        self.tree.heading(4, text="数")
        self.tree.heading(5, text="ドメイン")
        self.tree.heading(6, text="時間")
        
        # 列の幅
        self.tree.column(1, width=10)
        self.tree.column(2, width=100)
        self.tree.column(3, width=10)
        self.tree.column(4, width=10)
        self.tree.column(5, width=50)
        self.tree.column(6, width=50)

        # ツリービューの設置
        self.tree.pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
        # 選択されたTreeviewを取得するための関数をバインド
        self.tree.bind("<<TreeviewSelect>>", self.on_tree_select)

        self.load_pkl_insert_treeview()

        self.base.mainloop()

    def on_tree_select(self, event):
        self.selected = []
        for item in self.tree.selection():
            item_text = self.tree.item(item,"values")
            self.selected.append(item_text[0])
        print(self.selected)
        
    def query_click(self):
        # 検索語を取得
        query = self.entryBox.get()

        # ランキングを検索
        sr = Search(query, self.domain, self.filename)
        sr.search_term()
        
        # ツリービューの削除
        for i in self.tree.get_children():
            self.tree.delete(i)
            
        self.load_pkl_insert_treeview()
    
    
    def load_pkl_insert_treeview(self):
        # 保存したデータフレームの読み込み
        df = pd.read_pickle(self.filename)

        # 全データ挿入
        for i in range(df.index.stop):
            self.tree.insert("", "end", values=(i,df['SearchTerm'][i], df['Ranking'][i], df['Total'][i],
                                       df['TargetSite'][i], df['Date'][i]))
            
    def remove_click(self):
        # 保存したデータフレームの読み込み
        df = pd.read_pickle(self.filename)
        
        df.drop(df.index[self.selected], inplace=True)
        df.reset_index(inplace=True, drop=True)
        
        df.to_pickle(self.filename)
        
        # ツリービューの削除
        for i in self.tree.get_children():
            self.tree.delete(i)
            
        self.load_pkl_insert_treeview()
        
                

if __name__ == "__main__":
    app = RankCheckerGUI()

['18']


/opt/anaconda3/envs/golden-week/lib/python3.7/site-packages/pandas/core/indexes/base.py:3940: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/envs/golden-week/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-7-82b0a58a4364>", line 165, in remove_click
    df.drop(df.index[self.selected], inplace=True)
  File "/opt/anaconda3/envs/golden-week/lib/python3.7/site-packages/pandas/core/indexes/range.py", line 708, in __getitem__
    return super().__getitem__(key)
  File "/opt/anaconda3/envs/golden-week/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 3940, in __getitem__
    result = get